In [1]:
from stat_models.base import *
import pandas as pd
import numpy as np

# SBG

In [2]:
time = np.arange(0, 8, 1)
users = np.array([1000, 869, 743, 653, 593, 551, 517, 491])

array([0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
fit_sbg(time, users)

Shifted Beta Geometric Model with parameters: {'alpha': 0.6681058984632292, 'beta': 3.806247726362814}, log likelihood: -1611.158147411491

# NBD

In [2]:
nbd_data = pd.read_csv('class_examples/lecture_2_nbd.csv', index_col=0)

In [3]:
nbd_data_array = np.array([nbd_data.index, nbd_data.observations])

In [83]:
def fit_nbd(counts: np.ndarray, spikes:list=[], truncated:list=[], shift_start=0, right_censor_from=None)->Model:
    """
    Fits a negative binomial distribution to the provided counts data by maximizing the log likelihood.
    The function returns a Model instance with the fitted parameters r and p.

    Parameters:
    counts (np.ndarray): A 2D numpy array of counts and their associated people, optionally 3D if you want to add time
    spikes (list): A list of values for which there is a spike in the data
    truncated (list): A list values for which there are some "hard core nevers" in the data
    shift_start (int): The index at which to start the shift (default 0)
    right_censor_from (int): The index at which to start right censoring

    Raises:
    ValueError: If inputs are not 1D arrays, not of the same length, don't have at least 2 elements, contain negative values,
                or do not meet the increasing/decreasing requirement.

    Returns:
    Model: An instance of the Model class with fitted parameters
```"""
    # add a column for types
    if counts.shape[0] < 3:
        # Extend the counts array safely
        temp = np.zeros((3, counts.shape[1]))
        temp[:counts.shape[0], :] = counts
        temp[2, :] = 1
        counts = temp

    # Determine the full range of counts from 0 to max in the first row
    max_count = counts[0, :].max()
    full_range = np.arange(0, max_count + 1)

    # Initialize new_counts with 3 rows and full_range columns, third row defaults to 1
    new_counts = np.zeros((3, len(full_range)))
    new_counts[2, :] = 1  # Defaulting third row to 1

    # Populate new_counts with existing data
    for index in range(counts.shape[1]):
        value_index = int(counts[0, index])  # Ensure integer index
        if value_index < len(full_range):
            new_counts[:, value_index] = counts[:, index]

    counts = new_counts

    # Validate input arrays
    if len(counts[0]) < 2:
        raise ValueError("counts must have at least 2 elements")
    if not (np.all(counts[2, :] > 0)):
        raise ValueError("times must be positive")
    if not np.all(counts[0][:shift_start] == 0):
        raise ValueError("shift_start must be less than the first minimum in the counts data")

    def objective(params:list):
        # print(params)
        r, alpha = params[:2]
        spike_parameters = params[2:]
        probabilities = [] # store the probabilities associated with each individual observation of a count
        for idx in range(int(counts[0].max()+1)):
            if idx < shift_start:
                p = 0
                probabilities.append(p)
            elif idx - shift_start == 0: 
                p = (alpha/(alpha + counts[2][idx]))**r
                probabilities.append(p)
            else: 
                p = probabilities[idx-1] * (r+(idx-shift_start)-1)/(idx-shift_start)*(counts[2][idx]/(alpha + counts[2][idx]))
                probabilities.append(p)

        # calculate the probabilities for the spikes and truncated values
        truncated_probabilities = sum([probabilities[trunc] for trunc in truncated])
        spike_probabilities = sum([probabilities[spike] for spike in spikes])
        multiple = 1 - truncated_probabilities # allocate pro rata

        # adjust the probabilties  
        probabilities = [prob / multiple - spike_probabilities for prob in probabilities]
        for idx, spike_prob in enumerate(spikes): 
            probabilities[spike_prob] = spike_parameters[idx] + probabilities[spike_prob]
        # print(counts[1][right_censor_from])

        # right censor the data
        if right_censor_from is not None:
            right_censor_probabilties = sum(probabilities[:right_censor_from])
            # print(1-right_censor_probabilties)
            probabilities[right_censor_from] = 1 - right_censor_probabilties
            counts[1][right_censor_from] = counts[1][right_censor_from:].sum() # allocate all the remaining people in the right spike
            # print(counts[1][right_censor_from])
            counts[1][right_censor_from+1:] = 0 # set counts after that to 0
        
        # print([(p, idx) for p, idx in enumerate(probabilities)])
        # calculate log likelihood
        log_likelihood = counts[1] * np.log(probabilities)
        # print(log_likelihood)
        return -log_likelihood.sum()
    
    # initial guesses for alpha, r, spikes, and anti_spikes
    initial_guess = [1, 1]
    bounds = [(1e-5, None), (1e-5, None)]
    # append the spike probabilities 
    for _ in spikes:
        initial_guess.append(0)
        bounds.append((0, 1))

    result = minimize(objective, initial_guess, bounds=bounds)

    # Create and return a Model instance with the fitted parameters
    parameters={'r': result.x[0], 'alpha': result.x[1]}
    for idx, spike in enumerate(spikes): 
        parameters[f'spike_{spike}'] = result.x[idx+1]

    model = Model(
        parameters=parameters,
        log_likelihood= -result.fun,
        model_type=f"NBD, truncated: {truncated}, spikes: {spikes}, shift_start: {shift_start}, right_censor_from: {right_censor_from}"
    )
    return model

In [84]:
fit_nbd(nbd_data_array, right_censor_from=20)

NBD, truncated: [], spikes: [], shift_start: 0, right_censor_from: 20 Model with parameters: {'r': 0.16085371882848676, 'alpha': 0.12258690600364994}, log likelihood: -3842.081169500938

In [68]:
nbd_data_array

array([[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20],
       [2122,  340,  158,   98,   67,   49,   35,   28,   21,   17,   15,
          12,   10,    8,    7,    7,    5,    4,    5,    3,   30]])

In [9]:
temp = np.zeros((4, 2))
temp

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [10]:
temp[1] = 1
temp

array([[0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.]])